<img width="8%" alt="Pyvis.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Pyvis.png" style="border-radius: 15%">

# Pyvis - Visualize awesome notebooks graph network
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Pyvis+-+Visualize+awesome+notebooks+graph+network:+Error+short+description">Bug report</a>

**Tags:** #python #naas #asset #network #analytics

**Author:** [Jaime Dols Duxans](https://www.linkedin.com/in/duxans/)

**Last update:** 2023-12-01 (Created: 2023-11-30)

**Description:** With this notebook you can visualize all the awesome-notebooks templates as a graph network.

**References:** 
- [Pyvis documentation](https://pyvis.readthedocs.io/en/latest/tutorial.html)

## Input

### Import libraries

In [ ]:
import naas
try:
    from pyvis.network import Network
except:
    !pip install pyvis --upgrade
    from pyvis.network import Network
import pandas as pd
import requests

### Setup variables
- `github_url`: Stores the URL to a JSON file hosted on GitHub.
- `solver`: The gravity model used in the visualization (choose and test your favourite)
- `network_name`: Network name
- `height`: Network height
- `width`: Network width
- `bgcolor`: Network background color
- `font_color`: Network font color

In [ ]:
github_url = "https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/templates.json"
solver = 'force_atlas_2based'
network_name = "pyvis_awesome-notebooks"
height = "1400px"
width = "100%"
bgcolor = "#222222"
font_color = "white"

## Model

### Get templates from JSON

In [ ]:
def get_templates(url):
    res = requests.get(url)
    df =  pd.DataFrame(res.json())
    return df

df_temp = get_templates(github_url)
print("Templates:", len(df_temp))
df_temp.head(1)

### Initializing the Pyvis network

In [ ]:
net = Network(
    notebook=True,
    height=height,
    width=width,
    bgcolor=bgcolor,
    font_color=font_color
)

### Create Pyvis network

In [ ]:
# Physics solver. Choose one from Pyvis documentation  
net.force_atlas_2based()

# Pull data
tools = df_temp['tool'].tolist()
images = df_temp['image_url'].tolist()
notebooks = df_temp['notebook'].tolist()

# Create a dictionary of tools and images
tool_img = dict(zip(tools, images))

# Add tool nodes
for tool in set(tools):
    if tool == 'Naas':
        # We set Naas as the center node
        net.add_node(tool, title=tool, image=tool_img[tool], shape='image', mass=30, size=150, level=1, fixed=True, x=0, y=0, physics=False)
    elif tool == 'OpenAI':
        # OpenAI is a large node bouncing around very fast, so we let it be fixed
        net.add_node(tool, title=tool, image=tool_img[tool], shape='image', size=60, level=1, fixed=True, x=-1000, y=2000, physics=False)
    else:
        # All other tool nodes are of the same size and freely moving
        net.add_node(tool, title=tool, image=tool_img[tool], shape='image',size=60,level=1)

# Add notebook nodes
for notebook in set(notebooks):
    net.add_node(notebook, title=notebook, size=30, level=2, color='#4d94db')

# Add edges
for _, node_e in df_temp.iterrows():
    net.add_edge(node_e['tool'], node_e['notebook'], title=node_e['notebook_url'])
    
level_1_count = 0
level_2_count = 0
for node in net.nodes:
    if node['level'] == 1:
        level_1_count += 1
    else:
        level_2_count += 1

print("Number of tool nodes: ", level_1_count)
print("Number of notebook nodes: ", level_2_count)
print("Total number of nodes: ", len(net.nodes) , "\n")
print("Total number of edges: ", len(net.edges), "\n")

## Output

### Show results

In [ ]:
# Saving the network to a HTML file in your workdir (optional)
#net.write_html(f"{network_name}.html")

network = net.show(f"{network_name}.html")
network

### Share your output

In [ ]:
naas.asset.add(f"{network_name}.html", {"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete()